<a href="https://colab.research.google.com/github/Mktapphub/Edge-project/blob/main/Edge_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Minhajul khan")

Minhajul khan
